***Introduction***

This document provides a detailed overview of the methods and calculations essential for converting raw sensor currents into accurate gas concentration readings using Alphasense sensors. As leaders in gas detection technology, Alphasense provides robust solutions that are crucial for a variety of applications ranging from environmental monitoring to industrial safety. The focus here is to elucidate the specific equations and algorithms that are fundamental to the sensor data interpretation process, enabling users to transform electrical signals emitted by the sensors into meaningful data.

Contained within, you will find a comprehensive exploration of the mathematical models and conversion formulas as provided by Alphasense, designed to facilitate the accurate calculation of gas concentrations from the raw currents measured by the sensors. This documentation is intended to be a valuable resource for engineers, researchers, and professionals who rely on precise and dependable gas measurement techniques. Through an in-depth understanding of these conversion principles, users can ensure the effective application of Alphasense technology in their respective fields, thereby enhancing the reliability and accuracy of their gas detection systems.

---------------------------------------------------------------------

***Terms definitions***

- WEu = uncorrected raw WE output (mV - coming from BU)
- AEu = uncorrected raw AE output (mV - coming from BU)
- WEc = corrected WE output (Result Output)
- WEe = WE electronic offset on the AFE or ISB (mV - initially provided by Alphasense - source from AvSystem on runtime)
- AEe = AE electronic offset on the AFE or ISB (mV - initially provided by Alphasense - source from AvSystem on runtime)
- WEo = WE sensor zero, i.e. the sensor WE output in zero air (mV - initially provided by Alphasense - source from AvSystem on runtime)
- AEo = AE sensor zero, i.e. the sensor AE output in zero air (mV - initially provided by Alphasense - source from AvSystem on runtime)

- WET = Total WE zero offset
- AET = Total AE zero offset

- nT = temperature dependent correction factor for algorithm 1, refer to Table 3 for values
- kT = temperature dependent correction factor for algorithm 2, refer to Table 3 for values
- k'T = temperature dependent correction factor for algorithm 3, refer to Table 3 for values
- k''T = temperature dependent correction factor for algorithm 4, refer to Table 3 for values


----------------------------------------------------------------------

***Algorithms***

- Algorithm 1
    ```
    WEc = (WEu - WEe) - nT * (AEu - AEe)
    ```
- Algorithm 2
    ```
    WEc = (WEu - WEe) - kT * (WEo / AEo) * (AEu - AEe)
    ```
- Algorithm 3
    ```
    WEc = (WEu - WEe) - (WEo - AEo) - k'T * (AEu - AEe)
    ```
- Algorithm 4
    ```
    WEc = (WEu - WEe) - WEo - k''T
    ```

------------------------------------------------------------------
***Sensorbee Sensors***

- CO-B4
    - Type: B
    - Recommended Algorithm: Algorithm 1

- H2S-B4
    - Type: B
    - Recommended Algorithm: 1

- NO-B4
    - Type: B
    - Recommended Algorithm: 2

- NO2-B43F
    - Type: B
    - Recommended Algorithm: 1

- OX-B431
    - Type: B
    - Recommended Algorithm: 1

- SO2-B4
    - Type: B
    - Recommended Algorithm: 4


------------------------------------------------------
***Python Source Code Library***

In [15]:
import unittest
import numpy as np

class Converter:
  def __init__(self):
      self.algorithms = {
          1: {
              'parameters': ['WEu', 'WEe', 'AEu', 'AEe']
          },
          2: {
              'parameters': ['WEu', 'WEe', 'WEo', 'AEo', 'AEu', 'AEe']
          },
          3: {
              'parameters': ['WEu', 'WEe', 'WEo', 'AEo', 'AEu', 'AEe']
          },
          4: {
              'parameters': ['WEu', 'WEe', 'WEo']
          }
      }

      self.gasTypes = {
          'CO-B4': {
              'algo': 1,
              'alt_algo': 2, 
              'factors': [0.7, 0.7, 0.7, 0.7, 1, 3, 3.5, 4, 4.5]
          },
          'H2S-B4': {
              'algo': 1,
              'alt_algo': 2, 
              'factors': [-0.6, -0.6, 0.1, 0.8, -0.7, -2.5, -2.5, -2.2, -1.8]
          },
          'NO-B4': {
              'algo': 2,
              'alt_algo': 3, 
              'factors': [2.9, 2.9, 2.2, 1.8, 1.7, 1.6, 1.5, 1.4, 1.3]
          },
          'NO2-B43F': {
              'algo': 1,
              'alt_algo': 3, 
              'factors': [1.3, 1.3, 1.3, 1.3, 1, 0.6, 0.4, 0.2, -1.5]
          },
          'OX-B431': {
              'algo': 1,
              'alt_algo': 3, 
              'factors': [0.9, 0.9, 1, 1.3, 1.5, 1.7, 2, 2.5, 3.7]
          },
          'SO2-B4': {
              'algo': 4,
              'alt_algo': 2, 
              'factors': [-4, -4, -4, -4, -4, 0, 20, 140, 450]
          }
      }

  def generate_coeffs(self, factors):
    temperatures = np.array([-30, -20, -10, 0, 10, 20, 30, 40, 50]) #static temperature range base on the AlphaSense Zero background current temperature compensation factors
    degree = len(factors) - 1
    coefficients = np.polyfit(temperatures, factors, degree)
    return coefficients

  def evaluate_polynomial(self, factors, x):
    if len(factors) != 9:
        raise ValueError("Invalid number of factors. Exactly 9 factors are required.")

    coefficients = self.generate_coeffs(factors) #generate coefficients first using the temperature compensation factors 

    result = 0
    for index, coeff in enumerate(coefficients):
        power = len(coefficients) - 1 - index
        result += coeff * (x ** power)

    return result;
  
  def validate_parameters(self, check_list, parameters):
    total = len(check_list)
    ctr = 0

    for value in check_list:
      if value in parameters:
        ctr += 1

    return ctr == total
  
  def algo1(self, parameters, nT, wec_no2 = 0):
    return (parameters['WEu'] - (parameters['WEe'] + wec_no2)) - nT * (parameters['AEu'] - parameters['AEe'])
      
  def algo2(self, parameters, kt):
    return (parameters['WEu'] - parameters['WEe']) - kt * (parameters['WEo'] / parameters['AEo']) * (parameters['AEu'] - parameters['AEe'])

  def algo3(self, parameters, kt, wec_no2 = 0):
    return (parameters['WEu'] - (parameters['WEe'] + wec_no2)) - (parameters['WEo'] - parameters['AEo']) - kt * (parameters['AEu'] - parameters['AEe'])
  
  def algo4(self, parameters, kt):
    return (parameters['WEu'] - parameters['WEe']) - parameters['WEo'] - kt

  def convert_to_ppb(self, corrected, sensitivity):
    return corrected / sensitivity

  def convert_to_target_gas(self, stype, temperature, we_sensor, gain, parameters, no2_ppb = None, no2_sensitivity = None):
    algo = self.gasTypes[stype]['algo']
    alt_algo = self.gasTypes[stype]['alt_algo']
    factors = self.gasTypes[stype]['factors']
    sensitivity = (we_sensor * gain) / 1000
    validate_result = self.validate_parameters(self.algorithms[algo]['parameters'], parameters)
    calculation_methods = {
                1: self.algo1,
                2: self.algo2,
                3: self.algo3,
                4: self.algo4
            }
    if stype != 'OX-B431':
        # non-o3
        if validate_result:
            factor = self.evaluate_polynomial(factors, temperature)
            wec = calculation_methods[algo](parameters, factor)
            ppb = self.convert_to_ppb(wec, sensitivity)

            # if initial result is negative - use alternative algorithm
            if (ppb < 0):
                wec = calculation_methods[alt_algo](parameters, factor)
                ppb = self.convert_to_ppb(wec, sensitivity)
                return {'WEc': wec, 'correctionFactor': factor, 'ppb': ppb}
            else:
                return {'WEc': wec, 'correctionFactor': factor, 'ppb': ppb}
        else:
            raise ValueError("Incomplete list of parameters.")
    else: 
        # to extract O3 from OX-B431, the process requires no2 ppb, and no2 sensitivity
        # no2 ppb can be obtained using this function - just pass the necessary parameters for type NO2-B43F
        # no2 sensitivity is provided by alphasense
        if validate_result and no2_ppb is not None and no2_sensitivity is not None and (algo == 1 or algo == 3):
            factor = self.evaluate_polynomial(factors, temperature)
            wec_no2 = no2_ppb * no2_sensitivity
            wec = calculation_methods[algo](parameters, factor, wec_no2)
            ppb = self.convert_to_ppb(wec, sensitivity)

             # if initial result is negative - use alternative algorithm
            if (ppb < 0):
                wec = calculation_methods[alt_algo](parameters, factor, wec_no2)
                ppb = self.convert_to_ppb(wec, sensitivity)
                return {'WEc': wec, 'correctionFactor': factor, 'ppb': ppb}
            else:
                return {'WEc': wec, 'correctionFactor': factor, 'ppb': ppb}
        else:
            if not validate_result:
                raise ValueError("Incomplete list of parameters.")
            elif no2_ppb is None:
                raise ValueError("Missing No2 concentration.")
            elif no2_sensitivity is None:
                raise ValueError("Missing No2 sensitivity.")
            elif algo not in [1, 3]:
                raise ValueError("Invalid algorithm for O3.")


--------------------------------------

***Expected Parameters / Sources:***

***Electrodes***:
- ***WEe*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***AEe*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***WEo*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***AEo*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***WEu*** - to be sourced from AvSystem as raw voltage during runtime
- ***AEu*** - to be sourced from AvSystem as raw voltage during runtime

***Values / Settings*** 
- ***Temperature*** - to be sourced from AvSystem temperature object
- ***we_sensor (Sensitivity (nA/ppm))*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***gain*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime

If Sensor type is equal to OX-B431 then the following additional parameters are required:
- ***no2_ppb*** - no2 concentration from NO2-B43F. This can be obtained using the same function above, but with the needed parameters for NO2-B43F
- ***no2_sensitivity*** - provided by AlphaSense

---------------------------------------------------------------------------------------





***Test Environment Arguments/Parameters***

Enter Your Arguments/Parameters below (Update the values to see changes in the result):

In [18]:
WEu = 340.2908020019531
WEe = 355
AEu = 391.90704345703125
AEe = 352
AEo = 324
WEo = 433

sensor_type = 'CO-B4'
temperature = 22.734375
we_sensor = 545
gain = 0.800000011920929

no2_ppb = None
no2_sensitivity = None

***Test Environment***

In [19]:
class TestConverter(unittest.TestCase):
  def setUp(self):
    self.converter = Converter()

  def test_convert_to_target_gas(self):
    result = self.converter.convert_to_target_gas(sensor_type, temperature, we_sensor, gain, {
       'WEu': WEu, 
       'WEe': WEe, 
       'AEu': AEu, 
       'AEe': AEe,
       'AEo': AEo,
       'WEo': WEo
    }, no2_ppb, no2_sensitivity)
      
    print('result', result)    

unittest.main(argv=[''], verbosity=3, exit=False)

test_convert_to_target_gas (__main__.TestConverter.test_convert_to_target_gas) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


result {'WEc': -196.97221889246427, 'correctionFactor': 3.417481120707807, 'ppb': -451.7711375168567}
